<a href="https://colab.research.google.com/github/KHUSH445/ML_LAB/blob/main/cross_validation_bias_variance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving heart.csv to heart.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset (adjust path)
df = pd.read_csv("heart.csv")  # or the CSV you downloaded from Kaggle

# Basic inspection
print(df.head())
print(df.info())
print(df['target'].value_counts())  # target column (0 or 1) :contentReference[oaicite:4]{index=4}

# Separate features & target
X = df.drop(columns=['target'])
y = df['target'].values

# handle categorical features or missing values (if any)
X = pd.get_dummies(X, drop_first=True)
X.fillna(X.mean(), inplace=True)

# Convert to numpy (if required)
X = X.values.astype(float)
y = y.astype(int)

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold

model = LogisticRegression(max_iter=1000, random_state=42)

# k-fold cross validation
k = 5
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')

print(f"{k}-fold CV accuracy scores: {scores}")
print(f"Mean CV accuracy: {scores.mean():.4f}, Std: {scores.std():.4f}")

5-fold CV accuracy scores: [0.8597561  0.85365854 0.81097561 0.84146341 0.84756098]
Mean CV accuracy: 0.8427, Std: 0.0170


In [ ]:
model.fit(X_train_scaled, y_train)
y_test_pred = model.predict(X_test_scaled)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

test_acc = accuracy_score(y_test, y_test_pred)
print("Test set accuracy:", test_acc)
print(classification_report(y_test, y_test_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_test_pred))

Test set accuracy: 0.8097560975609757
              precision    recall  f1-score   support

           0       0.89      0.70      0.78       100
           1       0.76      0.91      0.83       105

    accuracy                           0.81       205
   macro avg       0.82      0.81      0.81       205
weighted avg       0.82      0.81      0.81       205

Confusion matrix:
 [[70 30]
 [ 9 96]]


In [ ]:
def bias_variance_estimate(model, X, y, n_rounds=30, test_size=0.2):
    from sklearn.model_selection import train_test_split
    predictions = []
    y_vals = None

    for i in range(n_rounds):
        X_tr, X_val, y_tr, y_val = train_test_split(
            X, y, test_size=test_size, stratify=y
        )
        scaler = StandardScaler()
        X_tr_s = scaler.fit_transform(X_tr)
        X_val_s = scaler.transform(X_val)

        model_clone = LogisticRegression(max_iter=1000, random_state=42)  # or clone
        model_clone.fit(X_tr_s, y_tr)
        preds = model_clone.predict(X_val_s)
        predictions.append(preds)
        y_vals = y_val

    predictions = np.array(predictions)  # shape (n_rounds, n_val_samples)
    mean_pred = np.mean(predictions, axis=0)  # average predicted class (float)
    # Convert mean_pred to binary decision if you like (e.g., >=0.5 → 1)

    # Estimate bias: how far average prediction is from true label
    bias = np.mean((mean_pred - y_vals) ** 2)

    # Estimate variance: average variability of predictions across runs
    variance = np.mean(np.var(predictions, axis=0))

    error = bias + variance
    return bias, variance, error

bias, variance, error = bias_variance_estimate(model, X_train_scaled, y_train)
print("Bias: ", bias)
print("Variance: ", variance)
print("Total approx error: ", error)

Bias:  0.24502032520325207
Variance:  0.23770325203252032
Total approx error:  0.4827235772357724


In [ ]:
from sklearn.tree import DecisionTreeClassifier

for depth in [1, 2, 4, 8, None]:
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')
    print(f"Depth {depth}: CV accuracy {scores.mean():.4f} ± {scores.std():.4f}")
    bias, var, err = bias_variance_estimate(model, X_train_scaled, y_train, n_rounds=20)
    print(f" → bias {bias:.4f}, variance {var:.4f}, error {err:.4f}\n")

Depth 1: CV accuracy 0.7695 ± 0.0430
 → bias 0.2479, variance 0.2345, error 0.4823

Depth 2: CV accuracy 0.7695 ± 0.0430
 → bias 0.2452, variance 0.2329, error 0.4780

Depth 4: CV accuracy 0.8524 ± 0.0151
 → bias 0.2555, variance 0.2323, error 0.4878

Depth 8: CV accuracy 0.9720 ± 0.0126
 → bias 0.2523, variance 0.2337, error 0.4860

Depth None: CV accuracy 0.9805 ± 0.0124
 → bias 0.2415, variance 0.2366, error 0.4780

